In [2]:
import warnings

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score

import funciones_auxiliares as aux

# warnings.filterwarnings('ignore')
sns.set_theme()
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
df = pd.read_csv('income-db.csv')
df_depurada = df.replace('?', np.nan)
df_depurada = df_depurada.dropna()
df_recodificada = aux.recodificar_enunciado_uno(df_depurada)
df_final = aux.one_hot_todas_las_categoricas(df_recodificada)
df_final.columns = df_final.columns.str.replace('-', '_')

# 1. Modelo completo

In [4]:
y = df_final['income']
X = df_final.drop(columns='income')
X_escalado = StandardScaler().fit_transform(X)

In [5]:
f1_acumulado = cross_val_score(LogisticRegression(),
                X=X_escalado,
                y=y,
                cv=10,
                scoring='f1')

In [6]:
f1_acumulado.mean()

0.6568000634243549

In [12]:
var_modelo_p = ["age",
"fnlwgt",
"educational_num",
"gender",
"capital_gain",
"capital_loss",
"hours_per_week",
"federal_gov",
"self_employed",
"divorced",
"married",
"separated",
"widowed",
"others",
"white_collar",
"Not_in_family",
"Other_relative",
"Own_child",
"Unmarried",
"Wife",
"Asian_Pac_Islander",
"White",
"america",
"asia",
"europe"]

y = df_final['income']
X_p = df_final[var_modelo_p]
X_p_escalado = StandardScaler().fit_transform(X_p)

In [13]:
f1_modelo_p = cross_val_score(LogisticRegression(),
                              X=X_p_escalado,
                              y=y,
                              cv=10,
                              scoring='f1')

In [16]:
f1_modelo_p.mean()

0.6567714461945567